**Day 5 Microsoft(Medium Level) https://lnkd.in/gXPMAD48 : Given a list of projects and employees mapped to each project, calculate by the amount of project budget allocated to each employee. The output should include the project title and the project budget rounded to the closest integer. Order your list by projects with the highest budget per employee first.**

In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/ms_projects', True)
dbutils.fs.rm('dbfs:/user/hive/warehouse/ms_emp_projects', True)

Out[5]: True

In [0]:
%sql
DROP TABLE IF EXISTS ms_projects;
DROP TABLE IF EXISTS ms_emp_projects;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ms_projects(id int, title varchar(15), budget int);
INSERT INTO ms_projects VALUES (1, 'Project1',  29498),(2, 'Project2',  32487),(3, 'Project3',  43909),(4, 'Project4',  15776),(5, 'Project5',  36268),(6, 'Project6',  41611),(7, 'Project7',  34003),(8, 'Project8',  49284),(9, 'Project9',  32341),(10, 'Project10',    47587);

CREATE TABLE IF NOT EXISTS ms_emp_projects(emp_id int, project_id int);
INSERT INTO ms_emp_projects VALUES (10592,  1),(10593,  2),(10594,  3),(10595,  4),(10596,  5),(10597,  6),(10598,  7),(10599,  8),(10600,  9),(10601,  10),(10602, 1),(10603, 2),(10604, 3),(10605, 4),(10606, 5),(10607, 6),(10608, 7),(10609, 8),(10610, 9),(10611, 10);

num_affected_rows,num_inserted_rows
20,20


In [0]:
%sql
SELECT p.title, round(p.budget/count(e.emp_id), 0) as budhet_per_project_per_emp FROM ms_projects p JOIN ms_emp_projects e ON p.id = e.project_id
GROUP BY p.title, p.budget ORDER BY budhet_per_project_per_emp DESC

title,budhet_per_project_per_emp
Project8,24642.0
Project10,23794.0
Project3,21955.0
Project6,20806.0
Project5,18134.0
Project7,17002.0
Project2,16244.0
Project9,16171.0
Project1,14749.0
Project4,7888.0


In [0]:
#------PySpark------------------------------------------------------------------------------------------
from pyspark.sql.functions import col, round

ms_projects_cols = ['id', 'title' , 'budget']
ms_projects_data = [(1, 'Project1',  29498),(2, 'Project2',  32487),(3, 'Project3',  43909),(4, 'Project4',  15776),(5, 'Project5',  36268),(6, 'Project6',  41611),(7, 'Project7',  34003),(8, 'Project8',  49284),(9, 'Project9',  32341),(10, 'Project10',    47587)]

ms_emp_projects_cols = ['emp_id', 'project_id']
ms_emp_projects_data = [(10592,  1),(10593,  2),(10594,  3),(10595,  4),(10596,  5),(10597,  6),(10598,  7),(10599,  8),(10600,  9),(10601,  10),(10602, 1),(10603, 2),(10604, 3),(10605, 4),(10606, 5),(10607, 6),(10608, 7),(10609, 8),(10610, 9),(10611, 10)]

ms_projects_df = spark.createDataFrame(data=ms_projects_data, schema=ms_projects_cols)
ms_emp_projects_df = spark.createDataFrame(data=ms_emp_projects_data, schema=ms_emp_projects_cols)
emp_cnt_per_project= ms_projects_df.join(ms_emp_projects_df, ms_projects_df.id == ms_emp_projects_df.project_id, 'inner')\
                                .groupBy('title', 'budget').count()
budget_per_project_per_emp_df = emp_cnt_per_project.select('title', round((col('budget')/col('count')), 0)\
                                .alias('budhet_per_project_per_emp'))
budget_per_project_per_emp_df.show()                 

+---------+--------------------------+
|    title|budhet_per_project_per_emp|
+---------+--------------------------+
| Project9|                   16171.0|
| Project7|                   17002.0|
| Project6|                   20806.0|
| Project4|                    7888.0|
| Project8|                   24642.0|
|Project10|                   23794.0|
| Project2|                   16244.0|
| Project3|                   21955.0|
| Project1|                   14749.0|
| Project5|                   18134.0|
+---------+--------------------------+



In [0]:
#------Pandas------------------------------------------------------------------------------------------
import pandas as pd
ms_projects_cols = ['id', 'title' , 'budget']
ms_projects_data = [(1, 'Project1',  29498),(2, 'Project2',  32487),(3, 'Project3',  43909),(4, 'Project4',  15776),(5, 'Project5',  36268),(6, 'Project6',  41611),(7, 'Project7',  34003),(8, 'Project8',  49284),(9, 'Project9',  32341),(10, 'Project10',    47587)]

ms_emp_projects_cols = ['emp_id', 'project_id']
ms_emp_projects_data = [(10592,  1),(10593,  2),(10594,  3),(10595,  4),(10596,  5),(10597,  6),(10598,  7),(10599,  8),(10600,  9),(10601,  10),(10602, 1),(10603, 2),(10604, 3),(10605, 4),(10606, 5),(10607, 6),(10608, 7),(10609, 8),(10610, 9),(10611, 10)]

pd_ms_projects = pd.DataFrame(data = ms_projects_data, columns=ms_projects_cols)
pd_emp_projects = pd.DataFrame(data=ms_emp_projects_data, columns=ms_emp_projects_cols)

pd_emp_cnt_per_project = pd_ms_projects.merge(pd_emp_projects, right_on= 'project_id', left_on='id',  how='inner')
pd_emp_cnt_per_project = pd_emp_cnt_per_project.groupby(['title', 'budget']).agg(emp_cnt = ('emp_id', 'count')).reset_index()
pd_emp_cnt_per_project['budget_per_project_per_emp'] = pd_emp_cnt_per_project.budget/pd_emp_cnt_per_project.emp_cnt
budget_per_project_per_emp_df = pd_emp_cnt_per_project[['title', 'budget_per_project_per_emp']]
budget_per_project_per_emp_df


,title,budget_per_project_per_emp
0,Project1,14749.0
1,Project10,23793.5
2,Project2,16243.5
3,Project3,21954.5
4,Project4,7888.0
5,Project5,18134.0
6,Project6,20805.5
7,Project7,17001.5
8,Project8,24642.0
9,Project9,16170.5


**Day 6 Airbnb(Medium Level) https://lnkd.in/gep8Td4w : Find the total number of available beds per hosts' nationality.
Output the nationality along with the corresponding total number of available beds. Sort records by the total available beds in descending order.**

In [0]:
%sql
CREATE TABLE airbnb_apartments(host_id int,apartment_id varchar(5),apartment_type varchar(10),n_beds int,n_bedrooms int,country varchar(20),city varchar(20));
INSERT INTO airbnb_apartments VALUES(0,'A1','Room',1,1,'USA','NewYork'),(0,'A2','Room',1,1,'USA','NewJersey'),(0,'A3','Room',1,1,'USA','NewJersey'),(1,'A4','Apartment',2,1,'USA','Houston'),(1,'A5','Apartment',2,1,'USA','LasVegas'),(3,'A7','Penthouse',3,3,'China','Tianjin'),(3,'A8','Penthouse',5,5,'China','Beijing'),(4,'A9','Apartment',2,1,'Mali','Bamako'),(5,'A10','Room',3,1,'Mali','Segou')

num_affected_rows,num_inserted_rows
9,9


In [0]:
%sql
CREATE TABLE airbnb_hosts(host_id int,nationality  varchar(15),gender varchar(5),age int);
INSERT INTO airbnb_hosts  VALUES(0,'USA','M',28),(1,'USA','F',29),(2,'China','F',31),(3,'China','M',24),(4,'Mali','M',30),(5,'Mali','F',30);

num_affected_rows,num_inserted_rows
6,6


In [0]:
%sql
SELECT  h.nationality, sum(a.n_beds) AS number_of_beds FROM airbnb_apartments a JOIN airbnb_hosts h ON a.host_id = h.host_id GROUP BY h.nationality ORDER BY number_of_beds DESC

nationality,number_of_beds
China,8
USA,7
Mali,5


In [0]:
#------PySpark------------------------------------------------------------------------------------------
from pyspark.sql.functions import col, desc
from pyspark.sql import functions as F
airbnb_apartments_cols =  ['host_id','apartment_id','apartment_type','n_beds','n_bedrooms','country' ,'city']
airbnb_apartments_data = [(0,'A1','Room',1,1,'USA','NewYork'),(0,'A2','Room',1,1,'USA','NewJersey'),(0,'A3','Room',1,1,'USA','NewJersey'),(1,'A4','Apartment',2,1,'USA','Houston'),(1,'A5','Apartment',2,1,'USA','LasVegas'),(3,'A7','Penthouse',3,3,'China','Tianjin'),(3,'A8','Penthouse',5,5,'China','Beijing'),(4,'A9','Apartment',2,1,'Mali','Bamako'),(5,'A10','Room',3,1,'Mali','Segou')]

airbnb_hosts_cols = ['host_id','nationality','gender','age']
airbnb_hosts_data = [(0,'USA','M',28),(1,'USA','F',29),(2,'China','F',31),(3,'China','M',24),(4,'Mali','M',30),(5,'Mali','F',30)]

airbnb_apartments_df = spark.createDataFrame(data=airbnb_apartments_data, schema=airbnb_apartments_cols)
airbnb_hosts_df = spark.createDataFrame(data=airbnb_hosts_data, schema=airbnb_hosts_cols)

nbeds_nationality_df = airbnb_hosts_df\
                        .join(airbnb_apartments_df, airbnb_hosts_df.host_id == airbnb_apartments_df.host_id, 'inner')\
                        .select('nationality', 'n_beds')\
                        .groupBy('nationality')\
                        .agg(F.sum('n_beds').alias('no_of_beds'))

nbeds_nationality_df.sort(desc('no_of_beds')).show()



+-----------+----------+
|nationality|no_of_beds|
+-----------+----------+
|      China|         8|
|        USA|         7|
|       Mali|         5|
+-----------+----------+



In [0]:
#------Pandas------------------------------------------------------------------------------------------
import pandas as pd

airbnb_apartments_cols =  ['host_id','apartment_id','apartment_type','n_beds','n_bedrooms','country' ,'city']
airbnb_apartments_data = [(0,'A1','Room',1,1,'USA','NewYork'),(0,'A2','Room',1,1,'USA','NewJersey'),(0,'A3','Room',1,1,'USA','NewJersey'),(1,'A4','Apartment',2,1,'USA','Houston'),(1,'A5','Apartment',2,1,'USA','LasVegas'),(3,'A7','Penthouse',3,3,'China','Tianjin'),(3,'A8','Penthouse',5,5,'China','Beijing'),(4,'A9','Apartment',2,1,'Mali','Bamako'),(5,'A10','Room',3,1,'Mali','Segou')]

airbnb_hosts_cols = ['host_id','nationality','gender','age']
airbnb_hosts_data = [(0,'USA','M',28),(1,'USA','F',29),(2,'China','F',31),(3,'China','M',24),(4,'Mali','M',30),(5,'Mali','F',30)]

pd_airbnb_apartments_df = pd.DataFrame(data=airbnb_apartments_data, columns=airbnb_apartments_cols)
pd_airbnb_hosts_df = pd.DataFrame(data=airbnb_hosts_data, columns=airbnb_hosts_cols)

nbeds_nationality_df = pd_airbnb_hosts_df.merge(pd_airbnb_apartments_df, on='host_id', how='inner')\
    .groupby('nationality').agg(no_of_beds = ('n_beds', 'sum')).reset_index()

nbeds_nationality_df.sort_values(by = 'no_of_beds', ascending=False).head()

,nationality,no_of_beds
0,China,8
2,USA,7
1,Mali,5


**Day 7 IBM(Hard Level) https://lnkd.in/gqx8mFNs: IBM is working on a new feature to analyze user purchasing behavior for all Fridays in the first quarter of the year. For each Friday separately, calculate the average amount users have spent per order. The output should contain the week number of that Friday and average amount spent.**

In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/user_purchases', True)

Out[2]: True

In [0]:
%sql
DROP TABLE IF EXISTS user_purchases 

In [0]:
%sql
CREATE TABLE IF NOT EXISTS user_purchases(user_id int, date date, amount_spent float, day_name varchar(15));

INSERT INTO user_purchases VALUES(1047,'2023-01-01',288,'Sunday'),(1099,'2023-01-04',803,'Wednesday'),(1055,'2023-01-07',546,'Saturday'),(1040,'2023-01-10',680,'Tuesday'),(1052,'2023-01-13',889,'Friday'),(1052,'2023-01-13',596,'Friday'),(1016,'2023-01-16',960,'Monday'),(1023,'2023-01-17',861,'Tuesday'),(1010,'2023-01-19',758,'Thursday'),(1013,'2023-01-19',346,'Thursday'),(1069,'2023-01-21',541,'Saturday'),(1030,'2023-01-22',175,'Sunday'),(1034,'2023-01-23',707,'Monday'),(1019,'2023-01-25',253,'Wednesday'),(1052,'2023-01-25',868,'Wednesday'),(1095,'2023-01-27',424,'Friday'),(1017,'2023-01-28',755,'Saturday'),(1010,'2023-01-29',615,'Sunday'),(1063,'2023-01-31',534,'Tuesday'),(1019,'2023-02-03',185,'Friday'),(1019,'2023-02-03',995,'Friday'),(1092,'2023-02-06',796,'Monday'),(1058,'2023-02-09',384,'Thursday'),(1055,'2023-02-12',319,'Sunday'),(1090,'2023-02-15',168,'Wednesday'),(1090,'2023-02-18',146,'Saturday'),(1062,'2023-02-21',193,'Tuesday'),(1023,'2023-02-24',259,'Friday');

num_affected_rows,num_inserted_rows
28,28


In [0]:
%sql
SELECT week, avg_amt_spent FROM
(SELECT 
    date, 
    sum(amount_spent) as total_amount,
    count(user_id) as user_cnt,
    sum(amount_spent)/count(user_id) as avg_amt_spent,
    DATE_PART('week', date) as week
    FROM user_purchases
    WHERE day_name = 'Friday' GROUP BY date ORDER BY week)

week,avg_amt_spent
2,742.5
4,424.0
5,590.0
8,259.0


In [0]:
#------PySpark------------------------------------------------------------------------------------------
from pyspark.sql.functions import weekofyear, col
from pyspark.sql import functions as F
user_purchases_cols = ['user_id', 'date', 'amount_spent', 'day_name']
user_purchases_data = [(1047,'2023-01-01',288,'Sunday'),(1099,'2023-01-04',803,'Wednesday'),(1055,'2023-01-07',546,'Saturday'),(1040,'2023-01-10',680,'Tuesday'),(1052,'2023-01-13',889,'Friday'),(1052,'2023-01-13',596,'Friday'),(1016,'2023-01-16',960,'Monday'),(1023,'2023-01-17',861,'Tuesday'),(1010,'2023-01-19',758,'Thursday'),(1013,'2023-01-19',346,'Thursday'),(1069,'2023-01-21',541,'Saturday'),(1030,'2023-01-22',175,'Sunday'),(1034,'2023-01-23',707,'Monday'),(1019,'2023-01-25',253,'Wednesday'),(1052,'2023-01-25',868,'Wednesday'),(1095,'2023-01-27',424,'Friday'),(1017,'2023-01-28',755,'Saturday'),(1010,'2023-01-29',615,'Sunday'),(1063,'2023-01-31',534,'Tuesday'),(1019,'2023-02-03',185,'Friday'),(1019,'2023-02-03',995,'Friday'),(1092,'2023-02-06',796,'Monday'),(1058,'2023-02-09',384,'Thursday'),(1055,'2023-02-12',319,'Sunday'),(1090,'2023-02-15',168,'Wednesday'),(1090,'2023-02-18',146,'Saturday'),(1062,'2023-02-21',193,'Tuesday'),(1023,'2023-02-24',259,'Friday')]

user_purchases_df = spark.createDataFrame(data=user_purchases_data, schema=user_purchases_cols)
user_purchases_df = user_purchases_df.withColumn('week', weekofyear(col('date')))\
    .filter(col('day_name') == 'Friday')\
        .groupBy('date', 'week')\
            .agg(F.sum(col('amount_spent')).alias('total_amount'),
                 F.count(col('user_id')).alias('user_cnt'))
            

user_purchases_df = user_purchases_df.select('week', (user_purchases_df.total_amount/user_purchases_df.user_cnt).alias('avg_spent'))
user_purchases_df.show()

+----+---------+
|week|avg_spent|
+----+---------+
|   2|    742.5|
|   4|    424.0|
|   5|    590.0|
|   8|    259.0|
+----+---------+



In [0]:
#------Pandas------------------------------------------------------------------------------------------
import pandas as pd

user_purchases_cols = ['user_id', 'date', 'amount_spent', 'day_name']
user_purchases_data = [(1047,'2023-01-01',288,'Sunday'),(1099,'2023-01-04',803,'Wednesday'),(1055,'2023-01-07',546,'Saturday'),(1040,'2023-01-10',680,'Tuesday'),(1052,'2023-01-13',889,'Friday'),(1052,'2023-01-13',596,'Friday'),(1016,'2023-01-16',960,'Monday'),(1023,'2023-01-17',861,'Tuesday'),(1010,'2023-01-19',758,'Thursday'),(1013,'2023-01-19',346,'Thursday'),(1069,'2023-01-21',541,'Saturday'),(1030,'2023-01-22',175,'Sunday'),(1034,'2023-01-23',707,'Monday'),(1019,'2023-01-25',253,'Wednesday'),(1052,'2023-01-25',868,'Wednesday'),(1095,'2023-01-27',424,'Friday'),(1017,'2023-01-28',755,'Saturday'),(1010,'2023-01-29',615,'Sunday'),(1063,'2023-01-31',534,'Tuesday'),(1019,'2023-02-03',185,'Friday'),(1019,'2023-02-03',995,'Friday'),(1092,'2023-02-06',796,'Monday'),(1058,'2023-02-09',384,'Thursday'),(1055,'2023-02-12',319,'Sunday'),(1090,'2023-02-15',168,'Wednesday'),(1090,'2023-02-18',146,'Saturday'),(1062,'2023-02-21',193,'Tuesday'),(1023,'2023-02-24',259,'Friday')]


pd_user_purchases_df = pd.DataFrame(data=user_purchases_data, columns=user_purchases_cols)
pd_user_purchases_df['week'] = pd.to_datetime(pd_user_purchases_df['date']).dt.isocalendar().week
pd_user_purchases_df = pd_user_purchases_df[pd_user_purchases_df['day_name'] == 'Friday']\
    .groupby(['date', 'week'])\
        .agg(total_amount = ('amount_spent', 'sum'),
             user_cnt = ('user_id', 'count')).reset_index()
pd_user_purchases_df['avg_spent'] =  pd_user_purchases_df.total_amount/pd_user_purchases_df.user_cnt
pd_user_purchases_df = pd_user_purchases_df.drop(columns=['date', 'total_amount', 'user_cnt'])

pd_user_purchases_df.head()

,week,avg_spent
0,2,742.5
1,4,424.0
2,5,590.0
3,8,259.0


**Day 8 Tesla(Medium Level) https://lnkd.in/gJwdGBtN: You are given a table of product launches by company by year. Write a query to count the net difference between the number of products companies launched in 2020 with the number of products companies launched in the previous year. Output the name of the companies and a net difference of net products released for 2020 compared to the previous year.**

In [0]:
%sql
CREATE TABLE car_launches(year int, company_name varchar(15), product_name varchar(30));

INSERT INTO car_launches VALUES(2019,'Toyota','Avalon'),(2019,'Toyota','Camry'),(2020,'Toyota','Corolla'),(2019,'Honda','Accord'),(2019,'Honda','Passport'),(2019,'Honda','CR-V'),(2020,'Honda','Pilot'),(2019,'Honda','Civic'),(2020,'Chevrolet','Trailblazer'),(2020,'Chevrolet','Trax'),(2019,'Chevrolet','Traverse'),(2020,'Chevrolet','Blazer'),(2019,'Ford','Figo'),(2020,'Ford','Aspire'),(2019,'Ford','Endeavour'),(2020,'Jeep','Wrangler')

num_affected_rows,num_inserted_rows
16,16


In [0]:
%sql
SELECT company_name, (current_year - previous_year) as net_difference 
FROM
    (SELECT
      company_name,
      sum(CASE WHEN year = '2020' THEN 1 ELSE 0 END) as current_year,
      sum(CASE WHEN year = '2019' THEN 1 ELSE 0 END) as previous_year
    FROM car_launches 
    GROUP BY company_name)
ORDER BY net_difference DESC

company_name,net_difference
Chevrolet,2
Jeep,1
Toyota,-1
Ford,-1
Honda,-3


In [0]:
#------PySpark------------------------------------------------------------------------------------------
from pyspark.sql.functions import when, col, desc
from pyspark.sql import functions as F
car_launches_cols = ['year', 'company_name', 'product_name']
car_launches_data = [(2019,'Toyota','Avalon'),(2019,'Toyota','Camry'),(2020,'Toyota','Corolla'),(2019,'Honda','Accord'),(2019,'Honda','Passport'),(2019,'Honda','CR-V'),(2020,'Honda','Pilot'),(2019,'Honda','Civic'),(2020,'Chevrolet','Trailblazer'),(2020,'Chevrolet','Trax'),(2019,'Chevrolet','Traverse'),(2020,'Chevrolet','Blazer'),(2019,'Ford','Figo'),(2020,'Ford','Aspire'),(2019,'Ford','Endeavour'),(2020,'Jeep','Wrangler')]

car_launches_df = spark.createDataFrame(data=car_launches_data, schema=car_launches_cols)
car_launches_df = car_launches_df.withColumn('current_year', when(col('year') == '2020', 1).otherwise(0))\
                                .withColumn('previous_year', when(col('year') == '2019', 1).otherwise(0))
car_launches_df = car_launches_df.groupBy('company_name').agg(F.sum('current_year').alias('current_year'),
                                                                      F.sum('previous_year').alias('previous_year'))
car_launches_df = car_launches_df\
                  .select(col('company_name'), (car_launches_df.current_year-car_launches_df.previous_year)\
                    .alias('net_difference'))
car_launches_df.sort(desc(col('net_difference'))).show()                             

+------------+--------------+
|company_name|net_difference|
+------------+--------------+
|   Chevrolet|             2|
|        Jeep|             1|
|      Toyota|            -1|
|        Ford|            -1|
|       Honda|            -3|
+------------+--------------+



In [0]:
#------Pandas------------------------------------------------------------------------------------------
import pandas as pd
car_launches_cols = ['year', 'company_name', 'product_name']
car_launches_data = [(2019,'Toyota','Avalon'),(2019,'Toyota','Camry'),(2020,'Toyota','Corolla'),(2019,'Honda','Accord'),(2019,'Honda','Passport'),(2019,'Honda','CR-V'),(2020,'Honda','Pilot'),(2019,'Honda','Civic'),(2020,'Chevrolet','Trailblazer'),(2020,'Chevrolet','Trax'),(2019,'Chevrolet','Traverse'),(2020,'Chevrolet','Blazer'),(2019,'Ford','Figo'),(2020,'Ford','Aspire'),(2019,'Ford','Endeavour'),(2020,'Jeep','Wrangler')]

pd_car_launches_df = pd.DataFrame(data=car_launches_data, columns=car_launches_cols)
pd_car_launches_df['current_year'] = pd_car_launches_df['year'].apply(lambda x: 1 if x == 2020 else 0)
pd_car_launches_df['previous_year'] = pd_car_launches_df['year'].apply(lambda x: 1 if x == 2019 else 0)
pd_car_launches_df = pd_car_launches_df.groupby(['company_name']).agg({'current_year':'sum', 'previous_year':'sum'}).reset_index()
pd_car_launches_df['net_difference'] = pd_car_launches_df.current_year - pd_car_launches_df.previous_year
pd_car_launches_df = pd_car_launches_df.drop(columns = ['current_year', 'previous_year'])
pd_car_launches_df.sort_values(by = 'net_difference', ascending=False).head()

,company_name,net_difference
0,Chevrolet,2
3,Jeep,1
1,Ford,-1
4,Toyota,-1
2,Honda,-3
